# Patching petitRADTRANS with the opac_mixer

In [1]:
import numpy as np
import glob
import os
from opac_mixer.read import ReadOpacChubb

import matplotlib.pyplot as plt

from opac_mixer.patches.prt import PatchedRadtrans
from petitRADTRANS import nat_cst as nc

from ipywidgets import interact, FloatSlider

from petitRADTRANS.poor_mans_nonequ_chem import interpol_abundances
from petitRADTRANS.physics import guillot_global

# %matplotlib widget
%matplotlib osx

## Setup

In [2]:
pressures = np.logspace(-6, 2, 10)

In [3]:
R='S1'
# R = '10'

In [4]:
mixmethod = 'deepset'
# mixmethod = 'aee'
# mixmethod = 'add'

In [5]:
weights = [np.load('data/mix_S1/ml_coeff_0.npy'), np.load('data/mix_S1/ml_coeff_1.npy')]

In [6]:
# use the default input and output scaling of the opac_mixer package (see opac_mixer.scalings)
input_scaling = None
inverse_output_scaling = None

In [7]:
base = f'{os.environ["pRT_input_data_path"]}/opacities/lines/corr_k'
files = glob.glob(os.path.join(base,f'*_R_{R}/*.h5'))

opac = ReadOpacChubb(files)
opac.read_opac()

In [8]:
linespecies = [f.split('/')[-1].split('.')[0] for f in files]
atmosphere_orig = PatchedRadtrans(line_species=linespecies, pressures=pressures, wlen_bords_micron=[(1e4/opac.bin_edges).min(), (1e4/opac.bin_edges).max()])
atmosphere = PatchedRadtrans(line_species=linespecies, pressures=pressures, wlen_bords_micron=[(1e4/opac.bin_edges).min(), (1e4/opac.bin_edges).max()])
atmosphere.setup_mixing(mixmethod=mixmethod, weights=weights, input_scaling=None, inverse_output_scaling=None)
atmosphere_orig.setup_mixing(mixmethod='rorr')


/Users/schneider/anaconda3/envs/ml/lib/python3.10/site-packages/petitRADTRANS/radtrans.py:100: FutureWarning: pRT_input_data_path was set by an environment variable. In a future update, the path to the petitRADTRANS input_data will be set within a .ini file that will be automatically generated into the user home directory (OS agnostic), inside a .petitradtrans directory
  warnings.warn(f"pRT_input_data_path was set by an environment variable. In a future update, the path to "


  Read line opacities of NH3_R_S1...
 Done.
  Read line opacities of CO_all_iso_Chubb_R_S1...
 Done.
  Read line opacities of TiO_all_Exomol_R_S1...
 Done.
  Read line opacities of H2O_Exomol_R_S1...
 Done.
  Read line opacities of PH3_R_S1...
 Done.
  Read line opacities of VO_R_S1...
 Done.
  Read line opacities of HCN_R_S1...
 Done.
  Read line opacities of Na_allard_R_S1...
 Done.
  Read line opacities of FeH_R_S1...
 Done.
  Read line opacities of SiO_R_S1...
 Done.
  Read line opacities of K_allard_R_S1...
 Done.
  Read line opacities of CO2_R_S1...
 Done.
  Read line opacities of H2S_R_S1...
 Done.
  Read line opacities of CH4_R_S1...
 Done.

  Read line opacities of NH3_R_S1...
 Done.
  Read line opacities of CO_all_iso_Chubb_R_S1...
 Done.
  Read line opacities of TiO_all_Exomol_R_S1...
 Done.
  Read line opacities of H2O_Exomol_R_S1...
 Done.
  Read line opacities of PH3_R_S1...
 Done.
  Read line opacities of VO_R_S1...
 Done.
  Read line opacities of HCN_R_S1...
 Done.
  Re

## top of the atmosphere (TOA) flux

In [14]:
fig_fl, ax_fl = plt.subplots(2,1, constrained_layout=True)

@interact(gamma=FloatSlider(min=0.0, max=10.0, step=0.2, value=0.4), T_int=FloatSlider(min=0.,max=1000,step=100., value=100.), T_equ=FloatSlider(min=0.,max=2000.,step=100., value=1000.))
def plot_flux_toa(gamma, T_int, T_equ):
    for axi in ax_fl:
        axi.cla()
    
    kappa_IR = 0.05      
    Tstar=6000
    Rstar=1.0*nc.r_sun
    semimajoraxis=0.01*nc.AU
    gravity = 1e1**2.45
    geometry='dayside_ave'

    # temperature = chubb.Tr
    temperature = guillot_global(pressures, kappa_IR, gamma, gravity, T_int, T_equ)

    COs = 0.55 * np.ones_like(pressures)
    FeHs = 0. * np.ones_like(pressures)

    mass_fractions = interpol_abundances(COs, \
                FeHs, \
                temperature, \
                pressures)

    for sp in linespecies:
        mass_fractions[sp] = mass_fractions.pop(sp.split('_')[0])            
    
    atmosphere_orig.calc_flux(temperature, mass_fractions, gravity, mass_fractions['MMW'], Rstar=Rstar, Tstar=Tstar, semimajoraxis=semimajoraxis, geometry=geometry)
    atmosphere.calc_flux(temperature, mass_fractions, gravity, mass_fractions['MMW'], Rstar=Rstar, Tstar=Tstar, semimajoraxis=semimajoraxis, geometry=geometry)

    bol_orig =np.sum(atmosphere_orig.flux*np.diff(atmosphere_orig.border_freqs))
    bol = np.sum(atmosphere.flux*np.diff(atmosphere.border_freqs))
    print('relative bolometric error on calculated flux:', (bol_orig-bol)/bol)

    ax_fl[0].plot(nc.c/atmosphere_orig.freq*1e4, atmosphere_orig.flux, label ='original')
    ax_fl[0].plot(nc.c/atmosphere.freq*1e4, atmosphere.flux, label ='emulator')
    ax_fl[0].set_xscale('log')
    ax_fl[0].set_xlabel(r'$\lambda$ / $\mu$m')
    ax_fl[0].set_ylabel(r'$F_\nu$ / cgs')
    ax_fl[1].loglog(temperature, pressures)
    ax_fl[1].set_ylabel(r'pressure / bar')
    ax_fl[1].set_xlabel('temperature')
    ax_fl[1].invert_yaxis()
    ax_fl[0].legend()


interactive(children=(FloatSlider(value=0.4, description='gamma', max=10.0, step=0.2), FloatSlider(value=100.0…

## Bolometric fluxes and heating rates

In [16]:
fig_flb, ax_flb = plt.subplots(1, 3, sharey= True, constrained_layout=True)

@interact(gamma=FloatSlider(min=0.0, max=10.0, step=0.2, value=0.4), T_int=FloatSlider(min=0.,max=1000,step=100., value=100.), T_equ=FloatSlider(min=0.,max=2000.,step=100., value=1000.))
def plot_flux_bol(gamma, T_int, T_equ):
    for axi in ax_flb:
        axi.cla()
    
    kappa_IR = 0.05      
    Tstar=6000
    Rstar=1.0*nc.r_sun
    semimajoraxis=0.01*nc.AU
    gravity = 1e1**2.45
    geometry='dayside_ave'

    # temperature = chubb.Tr
    temperature = guillot_global(pressures, kappa_IR, gamma, gravity, T_int, T_equ)

    COs = 0.55 * np.ones_like(pressures)
    FeHs = 0. * np.ones_like(pressures)

    mass_fractions = interpol_abundances(COs, \
                FeHs, \
                temperature, \
                pressures)

    for sp in linespecies:
        mass_fractions[sp] = mass_fractions.pop(sp.split('_')[0])            
    
    atmosphere_orig.calc_bolometric_flux(temperature, mass_fractions, gravity, mass_fractions['MMW'], Rstar=Rstar, Tstar=Tstar, semimajoraxis=semimajoraxis, geometry=geometry)
    atmosphere.calc_bolometric_flux(temperature, mass_fractions, gravity, mass_fractions['MMW'], Rstar=Rstar, Tstar=Tstar, semimajoraxis=semimajoraxis, geometry=geometry)

    p_e = atmosphere.press/1e6
    p_c = .5*(p_e[1:]+p_e[:-1])

    ax_flb[0].plot(atmosphere.flux_bol, p_e, label ='emulator')
    ax_flb[0].plot(atmosphere_orig.flux_bol, p_e, label ='original')
    ax_flb[1].plot(np.diff(atmosphere.flux_bol)/np.diff(p_e), p_c, label ='emulator')
    ax_flb[1].plot(np.diff(atmosphere_orig.flux_bol)/np.diff(p_e), p_c, label ='original')
    ax_flb[2].loglog(temperature, pressures)
    ax_flb[1].set_yscale('log')
    ax_flb[0].set_yscale('log')
    ax_flb[0].set_xscale('log')
    ax_flb[0].set_xlabel('F / cgs')
    ax_flb[1].set_xlabel('heatingrate / cgs')
    ax_flb[2].set_xlabel('temperature / K')
    ax_flb[0].set_ylabel('pressure / bar')
    # ax_fl[1].set_xscale('log')
    ax_flb[1].invert_yaxis()
    ax_flb[2].invert_yaxis()
    ax_flb[0].invert_yaxis()
    ax_flb[0].legend()

    print('times: orig, fake, speedup',atmosphere_orig.time_opa, atmosphere.time_opa, atmosphere_orig.time_opa/atmosphere.time_opa)


interactive(children=(FloatSlider(value=0.4, description='gamma', max=10.0, step=0.2), FloatSlider(value=100.0…

## Kappas

In [19]:
fig_kappa, ax_kappa = plt.subplots(10,1, figsize = (4,8), constrained_layout=True, sharex=True)

@interact(gamma=FloatSlider(min=0.0, max=10.0, step=0.2, value=0.4), T_int=FloatSlider(min=0.,max=1000,step=100., value=100.), T_equ=FloatSlider(min=0.,max=2000.,step=100., value=1000.))
def plot_eq_chem(gamma, T_int, T_equ):
    for axi in ax_kappa:
        axi.cla()  
    
    kappa_IR = 0.05      
    Tstar=6000
    Rstar=1.0*nc.r_sun
    semimajoraxis=0.01*nc.AU
    gravity = 1e1**2.45
    geometry='dayside_ave'

    # temperature = chubb.Tr
    temperature = guillot_global(pressures, kappa_IR, gamma, gravity, T_int, T_equ)

    COs = 0.55 * np.ones_like(pressures)
    FeHs = 0. * np.ones_like(pressures)

    mass_fractions = interpol_abundances(COs, \
                FeHs, \
                temperature, \
                pressures)

    for sp in linespecies:
        mass_fractions[sp] = mass_fractions.pop(sp.split('_')[0])   

    x_predict = np.empty((len(atmosphere.press),opac.ls+2))

    spec_names = {sp.split('_')[0]: sp for sp in atmosphere.line_species}

    for i, spec in enumerate(opac.spec):
        x_predict[:,i] = mass_fractions[spec_names[spec]]

    x_predict[:, -2] = atmosphere.press/1e6
    x_predict[:, -1] = temperature

    atmosphere.calc_flux(temperature, mass_fractions, gravity, mass_fractions['MMW'], Rstar=Rstar, Tstar=Tstar, semimajoraxis=semimajoraxis, geometry=geometry)
    line_struc_kappas_fake = atmosphere.line_struc_kappas[:,:,0,:]

    atmosphere_orig.calc_flux(temperature, mass_fractions, gravity, mass_fractions['MMW'], Rstar=Rstar, Tstar=Tstar, semimajoraxis=semimajoraxis, geometry=geometry)
    line_struc_kappas_orig = atmosphere_orig.line_struc_kappas[:,:,0,:]
    
    for i, p in enumerate(atmosphere.press):
        ax_kappa[i].plot(line_struc_kappas_fake[:,:,i].T.flatten(), ls = '-', alpha=0.4)
        ax_kappa[i].plot(line_struc_kappas_orig[:,:,i].T.flatten(), ls = '--', alpha=0.4)
    
        ax_kappa[i].set_yscale('log')
        ax_kappa[i].set_ylabel("$\kappa$ / cm2/g")

    ax_kappa[-1].set_xlabel(r"$\nu\cdot g$")

interactive(children=(FloatSlider(value=0.4, description='gamma', max=10.0, step=0.2), FloatSlider(value=100.0…